## Stable Diffusion Text-to-Image Generation on IPU

This notebook demonstrates how a stable diffusion inference pipeline can be run on Graphcore IPUs.

### Requirements

* An enabled Poplar SDK environment (or Paperspace account with access to the PyTorch IPU runtime)
* Additional dependencies installable via pip (done below)
* Access to the pretrained Stable-Diffusion-v1-5 checkpoint (done below)

In [ ]:
%%capture
!pip install -r requirements.txt
!pip install "ipywidgets>=7,<8"

Values for machine size and cache directories can be configured through environment variables or directly in the notebook:

In [ ]:
import os

pod_type = os.getenv("GRAPHCORE_POD_TYPE", "pod16")
executable_cache_dir = os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "/tmp/exe_cache/")

To download the pretrained Stable-Diffusion-v1-5 checkpoint, we must first authenticate to the Hugging Face Hub. Begin by creating a read access token on the [Hugging Face website](https://huggingface.co/settings/tokens) (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your read token:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

If you have not done so already, you will need to accept the User License on the [model page](https://huggingface.co/runwayml/stable-diffusion-v1-5).

### Pipeline Creation

We are now ready to import and run the pipeline.

In [ ]:
import torch

from ipu_models import IPUStableDiffusionPipeline

In [ ]:
pipe = IPUStableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", 
    revision="fp16", 
    torch_dtype=torch.float16,
    ipu_config={
        "executable_cache_dir": executable_cache_dir,
    }
)
pipe.enable_attention_slicing(slice_size=2)

In [ ]:
image_width = os.getenv("STABLE_DIFFUSION_TXT2IMG_DEFAULT_WIDTH", default=512)
image_height = os.getenv("STABLE_DIFFUSION_TXT2IMG_DEFAULT_HEIGHT", default=512)

We run a dummy generation step to trigger the one-time compilation process. This should take on the order of 15 minutes.

In [ ]:
pipe("apple", height=image_height, width=image_width, guidance_scale=7.5);

### Image Generation

Below you will find some example prompts. We encourage you to try your own!

In [ ]:
prompt = "a shiba inu in a zen garden, acrylic painting"
pipe(prompt, height=image_height, width=image_width, guidance_scale=7.5).images[0]

In [ ]:
prompt = "a photograph of an astronaut riding a horse"
pipe(prompt, height=image_height, width=image_width, guidance_scale=7.5).images[0]

In [ ]:
prompt = "the living room of a cozy wooden house with a fireplace"
pipe(prompt, height=image_height, width=image_width, guidance_scale=7.5).images[0]